# Attack Password with Correlation Power Analysis V (Template)

In [ ]:
%run '../helper_scripts/Metadata.ipynb'
print_metadata()

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Improving-the-code" data-toc-modified-id="Improving-the-code-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Improving the code</a></span></li><li><span><a href="#Basic-Setup" data-toc-modified-id="Basic-Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Basic Setup</a></span></li><li><span><a href="#Helper-Functions-for-Password-Attack" data-toc-modified-id="Helper-Functions-for-Password-Attack-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Helper Functions for Password Attack</a></span></li><li><span><a href="#MAD-attack" data-toc-modified-id="MAD-attack-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>MAD attack</a></span></li><li><span><a href="#Pearson-Correlation-Coefficient" data-toc-modified-id="Pearson-Correlation-Coefficient-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Pearson Correlation Coefficient</a></span><ul class="toc-item"><li><span><a href="#Definition" data-toc-modified-id="Definition-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Definition</a></span></li><li><span><a href="#Python-definition" data-toc-modified-id="Python-definition-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Python definition</a></span></li><li><span><a href="#Meaning-and-visualization" data-toc-modified-id="Meaning-and-visualization-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Meaning and visualization</a></span></li><li><span><a href="#Usage-for-attacks" data-toc-modified-id="Usage-for-attacks-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Usage for attacks</a></span></li></ul></li><li><span><a href="#CPA-password-attack" data-toc-modified-id="CPA-password-attack-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>CPA password attack</a></span></li><li><span><a href="#Notes" data-toc-modified-id="Notes-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Notes</a></span></li><li><span><a href="#Disconnect" data-toc-modified-id="Disconnect-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Disconnect</a></span></li></ul></div>

In this example we want to improve the password check again to beat an CPA attack.

## Improving the code


## Basic Setup

Define Variables

In [ ]:
%run "../helper_scripts/Init.ipynb"

Build target and upload

In [ ]:
TARGET = 'simpleserial-passwordcheck'
%store TARGET
%run "$HELPERSCRIPTS/Prepare.ipynb"

Import helper functions

In [ ]:
%run "$HELPERSCRIPTS/Setup_Generic.ipynb"

In [ ]:
scope.adc.samples = 200

## Helper Functions for Password Attack

In [ ]:
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool, Label

output_notebook()

In [ ]:
import warnings

def capture(command, data):
    scope.arm()

    target.simpleserial_write(command, data)

    ret = scope.capture()

    i = 0
    while not target.is_done():
        i += 1
        time.sleep(0.05)
        if i > 100:
            warnings.warn("Target did not finish operation")
            return None

    if ret:
        warnings.warn("Timeout happened during capture")
        return None

    return scope.get_last_trace()

In [ ]:
import numpy as np

HW = [bin(n).count("1") for n in range(0, 256)]

def hw(n):
    if isinstance(n, str):
        return HW[ord(n)]
    return HW[n]

hw_vec = np.vectorize(hw)

In [ ]:
import random
import tqdm
import numpy as np

def capture_random_pass(command='p', password_length=2, random_size=5000, password=None):
    traces = []
    textins = []
    passwds = []
    for _ in tqdm.tqdm_notebook(range(random_size)):
        data = bytes(random.choices(range(0, 256), k=2 * password_length))
        if password:
            data = password + data[password_length:]
        traces.append(capture(command, data))
        passwds.append(data[:password_length])
        textins.append(data[password_length:])
    return np.array(traces), passwds, textins

In [ ]:
traces, passwds, textins = capture_random_pass()

In [ ]:
traces_per_hw = [[] for _ in range(9)]
for trace, passwd, textin in zip(traces, passwds, textins):
    traces_per_hw[hw(passwd[0] ^ textin[0])].append(trace)
traces_per_hw = list(map(np.array, traces_per_hw))

In [ ]:
means_per_hw = [np.average(trace, 0) for trace in traces_per_hw]

In [ ]:
diffs = np.zeros(means_per_hw[0].shape[0])
for mean_i in means_per_hw:
    for mean_j in means_per_hw:
        diffs += np.abs(mean_i - mean_j)

In [ ]:
p = figure()
p.add_tools(CrosshairTool())
p.line(range(len(diffs)), diffs, color='blue', legend='Sum of diffs')
show(p)

In [ ]:
pois = np.argsort(diffs)[-10:]
pois

In [ ]:
def cov(x, y):
    # Find the covariance between two 1D lists (x and y).
    # Note that var(x) = cov(x, x)
    return np.cov(x, y)[0][1]

In [ ]:
template_mean = np.zeros((9, len(pois)))
template_cov = np.zeros((9, len(pois), len(pois)))

for ham in range(template_mean.shape[0]):
    for i in range(len(pois)):
        template_mean[ham, i] = means_per_hw[ham][pois[i]]
        for j in range(len(pois)):
            template_cov[ham, i, j] = cov(
                traces_per_hw[ham][:, pois[i]],
                traces_per_hw[ham][:, pois[j]],
            )
print('template_mean', template_mean.shape, '=\n', template_mean)
print('template_cov', template_cov.shape, '=\n', template_cov)

In [ ]:
traces, passwds, textins = capture_random_pass(random_size=10, password=b'if')

In [ ]:
from scipy.stats import multivariate_normal
# 2: Attack
# Running total of log P_k
P_k = np.zeros(256)
for j in range(len(traces)):
    # Grab key points and put them in a small matrix
    a = [traces[j][pois[i]] for i in range(len(pois))]
    
    # Test each key
    for k in range(256):
        # Find HW coming out of sbox
        guess_hw = hw(textins[j][0] ^ k)
    
        # Find p_{k,j}
        rv = multivariate_normal(template_mean[guess_hw], template_cov[guess_hw])
        p_kj = rv.logpdf(a)
   
        # Add it to running total
        P_k[k] += p_kj

    # Print our top 5 results so far
    # Best match on the right
    print(" ".join(["%02x"%j for j in P_k.argsort()[-5:]]))
    
guess = P_k.argsort()[-1]
print(hex(guess))

## Disconnect

In [ ]:
scope.dis()
target.dis()